import boto3
import bson
from bson.json_util import dumps, loads
import json
import os
from PIL import Image
import PIL
from io import BytesIO
import pandas as pd
import geopandas
import numpy as np
import feather
import h5py
import gc

pd.options.display.max_columns=999
pd.options.display.max_rows = 999

# import helper 
import sys
sys.path.append("/Users/xszpo/Google Drive/DataScience/Projects/201907_xFlat_AWS_Scrapy")
import helpers

# run scrapy settings
%run /Users/xszpo/Google\ Drive/DataScience/Projects/201907_xFlat_AWS_Scrapy/scraper/settings.py

LOCAL_DATA_PATH

# Load raw data

In [1]:
import os
import gc
from sklearn.model_selection import train_test_split
import numpy as np 
import scipy 
import keras 
import pandas as pd
from sklearn.pipeline import make_pipeline, make_union
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from itertools import compress
from itertools import chain
import unidecode
import re
import codecs
import geopandas
import json
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon

%matplotlib notebook

import sys
sys.path.append("/Users/xszpo/Google Drive/DataScience/Projects/201907_xFlat_AWS_Scrapy")
import helpers

# run scrapy settings
%run /Users/xszpo/Google\ Drive/DataScience/Projects/201907_xFlat_AWS_Scrapy/scraper/settings.py

pd.options.display.max_columns=999
pd.options.display.max_rows = 999

Using TensorFlow backend.


**Generator assumption**

**ID** - string that identifies a given sample of the dataset.     
**partition** -  dictionary with train/test split:    
    - in partition['train'] a list of training IDs
    - in partition['validation'] a list of validation IDs   
**labels** - dictionary where for each ID of the dataset, the associated label is given by labels[ID]

In [2]:
ID_list = [i[:-5] for i in os.listdir(LOCAL_DATA_PATH) if i.endswith(".bson")]

In [3]:
def read_bus_gpd(path, file, X = "X", Y="Y"):
    with codecs.open(os.path.join(path,file), "r", "utf-8") as file:
        bus_stops_json = json.loads(file.read())
    bus_stops_df = pd.DataFrame(bus_stops_json)
    bus_stops_gdf = geopandas.GeoDataFrame(
        bus_stops_df, geometry=geopandas.points_from_xy(bus_stops_df[X], bus_stops_df[Y]))
    return bus_stops_gdf


In [4]:
def load_raw_data(file_name_list, file_path):
    
    _dict = {}
    
    for name in file_name_list:
        # LOAD DATA
        _tmp = helpers.scraper.read_bson_local(file_path, name+".bson")
        _dict[name] = helpers.scraper.dict_except(_tmp,['gallery'])
        # red only the first image
        _dict[name]['img_gallery_strimg'] = _dict[name]['img_gallery_strimg'][:1]
    gc.collect()
    return _dict
    

In [5]:
def train_test_label_split(data_raw, train_val_test_size = [0.7,0.2,0.1], y_name = 'price'):
   
    train_val_test_size = np.cumsum(train_val_test_size/np.sum(train_val_test_size))
    
    
    _tmp_partition = {}
    _tmp_partition['train'] = []
    _tmp_partition['validation'] = []
    _tmp_partition['test'] = []
    _tmp_labels = {}

    for name in raw_data.keys():
        if helpers.scraper.digits_from_str(data_raw[name][y_name]) is not None:
            rand = np.random.rand()
            if rand <= train_val_test_size[0]:
                _tmp_partition['train'] += [name]
            elif train_val_test_size[0] < rand <= train_val_test_size[1]:
                _tmp_partition['validation'] += [name]
            else:
                _tmp_partition['test'] += [name]
            _tmp_labels[name] = helpers.scraper.digits_from_str(raw_data[name][y_name])
    gc.collect()
    return _tmp_partition, _tmp_labels

In [6]:
def transform_json_df(raw_json, partition = None):
    
    if partition:
        _tmp_list_train = []
        _tmp_list_validation = []
        _tmp_list_test = []
        file_list = partition['train']+partition['validation']+partition['test']
    else:
        _tmp_list = []
        file_list = list(raw_json.keys())
    
    for i,file in enumerate(file_list):
        
        # LOAD DATA
        _tmp = raw_json[file]
        _tmp = helpers.scraper.dict_except(_tmp,['img_gallery_strimg','gallery'])
        
        # PUT DATA INTO COLUMNS
        _tmp['file_name'] = file
        _tmp['GC_latitude'] = np.float32(_tmp['geo_coordinates']['latitude'])
        _tmp['GC_longitude'] = np.float32(_tmp['geo_coordinates']['longitude'])
        _tmp['GC_boundingbox'] = _tmp['geo_address_coordin']['@boundingbox']
        _tmp['GC_addr_house_number'] = _tmp['geo_address_text']['house_number'] if 'house_number' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_road'] = _tmp['geo_address_text']['road'] if 'road' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_neighbourhood'] = _tmp['geo_address_text']['neighbourhood'] if 'neighbourhood' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_suburb'] = _tmp['geo_address_text']['suburb'] if 'suburb' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_city'] = _tmp['geo_address_text']['city'] if 'city' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_county'] = _tmp['geo_address_text']['county'] if 'country' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_state'] = _tmp['geo_address_text']['state'] if 'state' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_postcode'] = _tmp['geo_address_text']['postcode'] if 'postcode' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_country'] = _tmp['geo_address_text']['country'] if 'country' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_country_code'] = _tmp['geo_address_text']['country_code'] if 'country_code' in _tmp['geo_address_text'] else None
        _tmp['additional_info'] =_tmp['additional_info'].split("|")
        _ = _tmp.pop('geo_coordinates')
        _ = _tmp.pop('geo_address_coordin') 
        _ = _tmp.pop('price_per_square')

        
        # MODIFY DATA
        _tmp['flat_size'] = helpers.scraper.digits_from_str(_tmp['flat_size']) if _tmp['flat_size'] is not None else None
        _tmp['price'] = helpers.scraper.digits_from_str(_tmp['price']) if _tmp['price'] is not None else None
        _tmp['price_m2'] = helpers.scraper.digits_from_str(_tmp['price_m2']) if _tmp['price_m2'] is not None else None
        _tmp['rooms'] = int(helpers.scraper.digits_from_str(_tmp['rooms'])) if _tmp['rooms'] is not None else None
        _tmp['floor_attic'] = 1 if _tmp['floor']=='poddasze' else 0
        _tmp['floor_basement'] = 1 if _tmp['floor']=='suterena' else 0
        _tmp['floor'] = np.float32(helpers.scraper.convert_floor(_tmp['floor'])) if isinstance(_tmp['floor'], (str)) else None
        _tmp['number_of_floors'] = np.float32(_tmp['number_of_floors']) if isinstance(_tmp['number_of_floors'], (str)) else None
        _tmp['year_of_building'] = np.float32(_tmp['year_of_building']) if isinstance(_tmp['year_of_building'], (str)) else None
        _tmp['rent_price'] = helpers.scraper.digits_from_str(_tmp['rent_price']) if _tmp['rent_price'] is not None else None
        
        # SAVE TO LIST
        if partition is None:
            _tmp_list += [_tmp]
        elif _tmp['file_name'] in partition['train']:
            _tmp_list_train += [_tmp]
        elif _tmp['file_name'] in partition['validation']:
            _tmp_list_validation += [_tmp]
        elif _tmp['file_name'] in partition['test']:
            _tmp_list_test += [_tmp]
        else:
            raise ValueError('could not find %s in %s' % (_tmp['file_name'],'partition'))
            
        
        columns = ['file_name','name','location','flat_size', 'rooms','floor','price',
                'price_m2', 'market', 'number_of_floors', 'floor_attic','floor_basement','building_type', 
                'building_material', 'widows_type', 'heating_type', 'year_of_building','finishing_stage', 
                'rent_price', 'property_form', 'available_from','description','additional_info',
                'GC_latitude','GC_longitude', 'GC_boundingbox', 'GC_addr_house_number','GC_addr_road', 
                'GC_addr_neighbourhood', 'GC_addr_suburb','GC_addr_city', 'GC_addr_county', 'GC_addr_state', 
                'GC_addr_postcode','GC_addr_country', 'GC_addr_country_code','url','main_url']
    
    if partition:
        
        return pd.DataFrame(_tmp_list_train)[columns], pd.DataFrame(_tmp_list_validation)[columns], pd.DataFrame(_tmp_list_test)[columns]
    else:
        return pd.DataFrame(_tmp_list)[columns]


In [7]:
class transformColList(BaseEstimator, TransformerMixin):
    """
    Transform columns of Data Frame 
    """

    def __init__(self, **kwargs):
        self.kwargs = kwargs
        
    def __correct_names(self, x):
        x = unidecode.unidecode(x)
        x = re.sub("[_.,/ ~!@#$%^&*()-+]","_",x)
        x = re.sub("_+","_",x)
        return x

    def fit(self, x, y=None):
        
        if type(x) == pd.core.frame.DataFrame:
            self.columns = x.columns
            self.cor_values = {}
            self.columnNames = []
            #df
            for col in self.columns:
                if type(x[col][0])==list:
                    self.cor_values[col] = set([(item,col+"_"+self.__correct_names(item)) for sublist in x[col].values for item in sublist if item != ''])
                    self.columnNames += [i[1] for i in self.cor_values[col]]
                else:
                    self.cor_values[col] = [(item,col+"_"+self.__correct_names(item)) for item in list(x[col].unique()) if item is not None]
                    self.columnNames += [i[1] for i in self.cor_values[col]]

        if type(x) == pd.core.series.Series:
            self.columns = [x.name]
            self.cor_values = {}
            #ser
    
            for col in self.columns:
                if type(x.to_list()[0])==list:
                    self.cor_values[col] = set([(item,col+"_"+self.__correct_names(item)) for sublist in x.values for item in sublist if item != ''])
                    self.columnNames = [i[1] for i in self.cor_values[col]]
                else:
                    self.cor_values[col] = [(item,col+"_"+self.__correct_names(item)) for item in list(x.unique()) if item is not None]
                    self.columnNames = [i[1] for i in self.cor_values[col]]

        return self 

    def get_feature_names(self):
        if hasattr(self, "columnNames"):
            return self.columnNames
        else:
            return None

    def transform(self, x):
                
        data = {}
        for i in self.columns:
            data[i] = []
            
        if type(x) == pd.core.frame.DataFrame:
            #df
            for col in self.columns:
                if type(x[col][0])==list:
                    
                    _dict = {}
                    
                    for i in self.cor_values[col]:
                        _dict[i[1]] = 0

                    for i,row in enumerate(x[col].tolist()):
                        _dict_tmp = _dict.copy()
                        for val in self.cor_values[col]:
                            if val[0] in row:
                                _dict_tmp[val[1]] = 1
                            else:
                                _dict_tmp[val[1]] = 0
                        data[col] += [_dict_tmp]
                    
                else:
                                       
                    _dict = {}
                    
                    for i in self.cor_values[col]:
                        _dict[i[1]] = 0

                    for row in x[col].tolist():
                        _dict_tmp = _dict.copy()
                        for val in self.cor_values[col]:
                            if val[0] == row:
                                _dict_tmp[val[1]] = 1
                            else:
                                _dict_tmp[val[1]] = 0
                        data[col] += [_dict_tmp]         
                
            return pd.concat([pd.DataFrame(data[i]) for i in data],axis=1)
                    
                                      
        if type(x) == pd.core.series.Series:
            #series
            for col in self.columns:
                if type(x[0])==list:
                    
                    _dict = {}
                    
                    for i in self.cor_values[col]:
                        _dict[i[1]] = 0

                    for i,row in enumerate(x.tolist()):
                        _dict_tmp = _dict.copy()
                        for val in self.cor_values[col]:
                            if val[0] in row:
                                _dict_tmp[val[1]] = 1
                            else:
                                _dict_tmp[val[1]] = 0
                        data[col] += [_dict_tmp]
                    
                else:
                                       
                    _dict = {}
                    
                    for i in self.cor_values[col]:
                        _dict[i[1]] = 0

                    for row in x.tolist():
                        _dict_tmp = _dict.copy()
                        for val in self.cor_values[col]:
                            if val[0] == row:
                                _dict_tmp[val[1]] = 1
                            else:
                                _dict_tmp[val[1]] = 0
                        data[col] += [_dict_tmp]         
                
            return pd.concat([pd.DataFrame(data[i]) for i in data],axis=1)
    

In [14]:
def find_metro(metro_gpd, df, dist = 400/111320):
    
    df_temp = geopandas.GeoDataFrame(
        df[['file_name','location']], geometry=geopandas.points_from_xy(df.GC_longitude, df.GC_latitude)
    )
    df_temp_find_metro = geopandas.sjoin(
        df_temp.assign(geometry=lambda x: x.buffer(dist)),metro_gpd,how="inner", op='intersects').assign(
                ok_loc = lambda x: x['location'].apply(lambda xx: len(xx.split(","))>2)).query("ok_loc == True")
    df_temp_find_metro = df_temp_find_metro.rename(columns={"name": "metro_name"})[['file_name','metro_name']]
    duplic = df_temp_find_metro.duplicated('file_name')
    df_temp_find_metro = df_temp_find_metro.loc[[not(i) for i in duplic.tolist()]].reset_index(drop=True)
    
    
    file_name_list = df_temp_find_metro.file_name.tolist()
    
    df_result = pd.merge(df, df_temp_find_metro, on='file_name', how='left').fillna({'metro_name':'brak'})
    
    return df_result    

In [9]:
%%time
raw_data = load_raw_data(ID_list, LOCAL_DATA_PATH)

CPU times: user 17.1 s, sys: 3.48 s, total: 20.6 s
Wall time: 24.9 s


In [12]:
gpd = read_bus_gpd(path=LOCAL_DATA_PATH_DICT, file='bus_stops.json')
gpd_metro =gpd[gpd.name.str.match('Metro')]

In [10]:
partition, labels = train_test_label_split(raw_data,train_val_test_size = [0.7,0.2,0.1])

In [11]:
df_train, df_valid, df_test = transform_json_df(raw_data, partition)

In [15]:
df_train = find_metro(gpd_metro,df_train)
df_valid = find_metro(gpd_metro,df_valid)
df_test = find_metro(gpd_metro,df_test)


In [16]:
df_train.columns

Index(['file_name', 'name', 'location', 'flat_size', 'rooms', 'floor', 'price',
       'price_m2', 'market', 'number_of_floors', 'floor_attic',
       'floor_basement', 'building_type', 'building_material', 'widows_type',
       'heating_type', 'year_of_building', 'finishing_stage', 'rent_price',
       'property_form', 'available_from', 'description', 'additional_info',
       'GC_latitude', 'GC_longitude', 'GC_boundingbox', 'GC_addr_house_number',
       'GC_addr_road', 'GC_addr_neighbourhood', 'GC_addr_suburb',
       'GC_addr_city', 'GC_addr_county', 'GC_addr_state', 'GC_addr_postcode',
       'GC_addr_country', 'GC_addr_country_code', 'url', 'main_url',
       'metro_name'],
      dtype='object')

In [17]:
col_numeric = ['flat_size', 'floor_attic','floor_basement', 'rooms', 'floor', 'number_of_floors','year_of_building','GC_latitude','GC_llongitude']
col_cat = ['market' ,'building_type', 'building_material', 'widows_type',
       'heating_type', 'finishing_stage', 'property_form','metro_name']
cos=['GC_latitude', 'GC_longitude',
       'GC_boundingbox', 'GC_addr_house_number', 'GC_addr_road',
       'GC_addr_neighbourhood', 'GC_addr_suburb', 'GC_addr_city',
       'GC_addr_county', 'GC_addr_state', 'GC_addr_postcode',
       'GC_addr_country', 'GC_addr_country_code', 'url', 'main_url']
col_text = ['name', 'location','description']
col_dat = ['available_from']

In [18]:
tr = transformColList()
tr.fit_transform(df_train[col_cat])
tr.transform(df_valid[col_cat])
tr.transform(df_test[col_cat])


,market_wtorny,market_pierwotny,building_type_blok,building_type_kamienica,building_type_apartamentowiec,building_type_dom_wolnostojacy,building_type_plomba,building_type_szeregowiec,building_type_loft,building_material_wielka_plyta,building_material_cegla,building_material_zelbet,building_material_inne,building_material_pustak,building_material_beton,building_material_silikat,building_material_beton_komorkowy,building_material_drewno,building_material_keramzyt,widows_type_plastikowe,widows_type_drewniane,widows_type_aluminiowe,heating_type_miejskie,heating_type_inne,heating_type_gazowe,heating_type_kotlownia,heating_type_elektryczne,heating_type_piece_kaflowe,finishing_stage_do_zamieszkania,finishing_stage_do_wykonczenia,finishing_stage_do_remontu,property_form_spoldzielcze_wl_z_KW,property_form_pelna_wlasnosc,property_form_spoldzielcze_wlasnosciowe,property_form_udzial,metro_name_brak,metro_name_Metro_Swietokrzyska_,metro_name_Metro_Raclawicka_,metro_name_Metro_Kabaty_,metro_name_Metro_Wierzbno_,metro_name_Metro_Mlociny_,metro_name_Metro_Stare_Bielany_,metro_name_Metro_Politechnika_,metro_name_Metro_Stadion_Narodowy_,metro_name_Metro_Centrum_Nauki_Kopernik_,metro_name_Metro_Natolin_,metro_name_Metro_Pole_Mokotowskie_,metro_name_Metro_Marymont_,metro_name_Metro_Wilanowska_,metro_name_Metro_Ratusz_Arsenal_,metro_name_Metro_Imielin_,metro_name_Metro_Ursynow_,metro_name_Metro_Stoklosy_,metro_name_Metro_Wawrzyszew_,metro_name_Metro_Sluzew_
0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1563,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1564,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1565,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
pipe_numeric_cat = make_pipeline(
    ColumnTransformer([
        ('numeric', helpers.DecisionTreeDiscretizer_DF(bins = manual_bins,
            max_depth = 3, min_samples_prc_leaf=0.05),features_all_numeric),
        ('category',helpers.PassThroughOrReplace(fillna=True, replace_dict=dict_repl),features_all_category)
    ]),
    WOEEncoder(random_state=seed),
    LogisticRegression(random_state=seed, C=0.05, penalty='l1')
    )

In [ ]:
import numpy as np
import keras

class DataGenerator(keras.utils.Sequence):
    """
    Generates data for Keras
    https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
    """
    def __init__(self, list_IDs, labels, raw_json, batch_size=32, shuffle=True):
        'Initialization'
        self.list_IDs = list_IDs
        self.labels = labels
        self.raw_json = raw_json
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        #self.dim = dim
        #self.n_channels = n_channels
        #self.n_classes = n_classes

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))
    
    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def json2numpy(self):
        
        
            
    #def __data_generation(self, list_IDs_temp):
    def data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = np.load('data/' + ID + '.npy')

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)


In [ ]:
dg = DataGenerator(list_IDs = ID_list, labels = labels, raw_json = raw_data)
dg.data_generation(ID_list[:10])

# Load Data Frame

In [ ]:
# load data frame

def load_data(file_name_list, file_path):
    
    _tmp_list = [None for i in range(len(file_name_list))]
    
    for i,file in enumerate(file_name_list):
        
        # LOAD DATA
        _tmp = helpers.scraper.read_bson_local(file_path, file)
        _tmp = helpers.scraper.dict_except(_tmp,['img_gallery_strimg','gallery'])
        
        # PUT DATA INTO COLUMNS
        _tmp['file_name'] = file
        _tmp['GC_latitude'] = _tmp['geo_coordinates']['latitude']
        _tmp['GC_llongitude'] = _tmp['geo_coordinates']['longitude']
        _tmp['GC_boundingbox'] = _tmp['geo_address_coordin']['@boundingbox']
        _tmp['GC_addr_house_number'] = _tmp['geo_address_text']['house_number'] if 'house_number' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_road'] = _tmp['geo_address_text']['road'] if 'road' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_neighbourhood'] = _tmp['geo_address_text']['neighbourhood'] if 'neighbourhood' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_suburb'] = _tmp['geo_address_text']['suburb'] if 'suburb' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_city'] = _tmp['geo_address_text']['city'] if 'city' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_county'] = _tmp['geo_address_text']['county'] if 'country' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_state'] = _tmp['geo_address_text']['state'] if 'state' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_postcode'] = _tmp['geo_address_text']['postcode'] if 'postcode' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_country'] = _tmp['geo_address_text']['country'] if 'country' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_country_code'] = _tmp['geo_address_text']['country_code'] if 'country_code' in _tmp['geo_address_text'] else None
        _ = _tmp.pop('geo_coordinates')
        _ = _tmp.pop('geo_address_coordin')
        _ = _tmp.pop('price_per_square')

        
        # MODIFY DATA
        _tmp['flat_size'] = helpers.scraper.digits_from_str(_tmp['flat_size']) if _tmp['flat_size'] is not None else None
        _tmp['price'] = helpers.scraper.digits_from_str(_tmp['price']) if _tmp['price'] is not None else None
        _tmp['price_m2'] = helpers.scraper.digits_from_str(_tmp['price_m2']) if _tmp['price_m2'] is not None else None
        _tmp['rooms'] = int(helpers.scraper.digits_from_str(_tmp['rooms'])) if _tmp['rooms'] is not None else None
        _tmp['floor_attic'] = 1 if _tmp['floor']=='poddasze' else 0
        _tmp['floor_basement'] = 1 if _tmp['floor']=='suterena' else 0
        _tmp['floor'] = np.float32(helpers.scraper.convert_floor(_tmp['floor'])) if isinstance(_tmp['floor'], (str)) else None
        _tmp['number_of_floors'] = np.float32(_tmp['number_of_floors']) if isinstance(_tmp['number_of_floors'], (str)) else None
        _tmp['year_of_building'] = np.float32(_tmp['year_of_building']) if isinstance(_tmp['year_of_building'], (str)) else None
        _tmp['rent_price'] = helpers.scraper.digits_from_str(_tmp['rent_price']) if _tmp['rent_price'] is not None else None
        
        # SAVE TO LIST
        _tmp_list[i] = _tmp
        
        columns = ['offer_id','tracking_id','name','location','flat_size', 'rooms','floor','price',
                'price_m2', 'market', 'number_of_floors', 'floor_attic','floor_basement','building_type', 
                'building_material', 'widows_type', 'heating_type', 'year_of_building','finishing_stage', 
                'rent_price', 'property_form', 'available_from','description','additional_info',
                'GC_latitude','GC_llongitude', 'GC_boundingbox', 'GC_addr_house_number','GC_addr_road', 
                'GC_addr_neighbourhood', 'GC_addr_suburb','GC_addr_city', 'GC_addr_county', 'GC_addr_state', 
                'GC_addr_postcode','GC_addr_country', 'GC_addr_country_code','url','main_url']
    
    return pd.DataFrame(_tmp_list)[columns]


In [ ]:
data_df = load_data(file_list,LOCAL_DATA_PATH)


In [ ]:
feather.write_dataframe(data_df, os.path.join(LOCAL_DATA_PATH_PREP,'train_data.feather'))

# Load Images

In [ ]:
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import numpy as np

In [ ]:
%%time

def load_data(file_name_list, file_path, img_size = (224, 224)):
    
    _tmp_list = [None for i in range(len(file_name_list))]
    
    for i,file in enumerate(file_name_list):

        # LOAD DATA
        _tmp = helpers.scraper.read_bson_local(file_path, file)
        _tmp = helpers.scraper.dict_except(_tmp,include_keys=['price','img_gallery_strimg'])
        
        # PUT DATA INTO COLUMNS
        _tmp['file_name'] = file
        _tmp['price'] = helpers.scraper.digits_from_str(_tmp['price']) if _tmp['price'] is not None else None
        _tmp['photo_1'] = helpers.scraper.open_img_from_str(_tmp['img_gallery_strimg'][0]).resize(img_size) if len(_tmp['img_gallery_strimg'])>0 else None
        _tmp['photo_2'] = helpers.scraper.open_img_from_str(_tmp['img_gallery_strimg'][1]).resize(img_size) if len(_tmp['img_gallery_strimg'])>1 is not None else None
        _tmp['photo_3'] = helpers.scraper.open_img_from_str(_tmp['img_gallery_strimg'][2]).resize(img_size) if len(_tmp['img_gallery_strimg'])>2 is not None else None
        _ = _tmp.pop('img_gallery_strimg')
        
        # SAVE TO LIST
        _tmp_list[i] = _tmp
        
    return _tmp_list

photo = load_data(file_list,LOCAL_DATA_PATH)


In [ ]:
name = []
price = []

for i in photo:
    name += [i['file_name']]
    price += [i['price']]
    

In [ ]:
name = np.array(name).reshape(-1,1)
price = np.array(price).reshape(-1,1)

In [ ]:
%%time

array_photo_1 = []
array_photo_2 = []
array_photo_3 = []

for i in photo:
    array_photo_1 += [image.img_to_array(i['photo_1']) if i['photo_1'] is not None else np.zeros((224, 224, 3))]
    array_photo_2 += [image.img_to_array(i['photo_2']) if i['photo_2'] is not None else np.zeros((224, 224, 3))]
    array_photo_3 += [image.img_to_array(i['photo_3']) if i['photo_3'] is not None else np.zeros((224, 224, 3))]


In [ ]:
import gc

del(photo)
gc.collect()


In [ ]:
array_photo_1 = np.concatenate(array_photo_1).reshape(-1,224,224,3)

In [ ]:
array_photo_2 = np.concatenate(array_photo_2).reshape(-1,224,224,3)

In [ ]:
array_photo_3 = np.concatenate(array_photo_3).reshape(-1,224,224,3)

In [ ]:
import gc
gc.collect()

https://keras.io/getting-started/faq/#how-can-i-use-hdf5-inputs-with-keras    
http://tdeboissiere.github.io/h5py-vs-npz.html

%%time

with h5py.File(os.path.join(LOCAL_DATA_PATH_PREP,'array_photo.h5'), "w") as hf:
    dset = hf.create_dataset("array_photo_1", data=array_photo_1, compression="gzip")
    dset = hf.create_dataset("array_photo_2", data=array_photo_2, compression="gzip")
    dset = hf.create_dataset("array_photo_3", data=array_photo_3, compression="gzip")
    dset = hf.create_dataset("name", data=name, compression="gzip", compression_opts=9)
    dset = hf.create_dataset("price", data=price, compression="gzip", compression_opts=9)

In [ ]:
np.savez_compressed(os.path.join(LOCAL_DATA_PATH_PREP,'array_photo'), 
                    a1=array_photo_1, a2=array_photo_2, a3=array_photo_3)